In [ ]:
import sys
sys.path.extend([ '../', '../../', '../../../'])
import torch
import dataloader as dl
from args import Args
# from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from model import MAG_ConEn, MAG_NetEn, EdgePredictor, MAG_classify
from train import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.pubmed()
torch.cuda.empty_cache()

Current device: NVIDIA A100-SXM4-80GB


In [2]:
data = torch.load('../../data/pubmed_data.pt', weights_only=False)
print(data)

HeteroData(
  g={
    num_nodes=13561,
    x=[13561, 200],
  },
  d={
    num_nodes=20163,
    x=[20163, 200],
    y=[454, 2],
  },
  c={
    num_nodes=26522,
    x=[26522, 200],
  },
  s={
    num_nodes=2863,
    x=[2863, 200],
  },
  (g, walk, g)={ edge_index=[2, 95527] },
  (g, walk, d)={ edge_index=[2, 96007] },
  (g, walk, c)={ edge_index=[2, 121066] },
  (g, walk, s)={ edge_index=[2, 27554] },
  (d, walk, g)={ edge_index=[2, 136561] },
  (d, walk, d)={ edge_index=[2, 144073] },
  (d, walk, c)={ edge_index=[2, 178599] },
  (d, walk, s)={ edge_index=[2, 40564] },
  (c, walk, g)={ edge_index=[2, 180817] },
  (c, walk, d)={ edge_index=[2, 187501] },
  (c, walk, c)={ edge_index=[2, 239230] },
  (c, walk, s)={ edge_index=[2, 53710] },
  (s, walk, g)={ edge_index=[2, 19256] },
  (s, walk, d)={ edge_index=[2, 19906] },
  (s, walk, c)={ edge_index=[2, 25073] },
  (s, walk, s)={ edge_index=[2, 6571] }
)


In [3]:
print(data['d'].y[:10,:])
args.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['g']['x'] = data['g']['x'].to(device)
data['d']['x'] = data['d']['x'].to(device)
data['c']['x'] = data['c']['x'].to(device)
data['s']['x'] = data['s']['x'].to(device)

# Send all y tensors to the device
data['d']['y'] = data['d']['y'].to(device)

data['g', 'walk', 'g']['edge_index'] = data['g', 'walk', 'g']['edge_index'].to(device) 
data['g', 'walk', 'd']['edge_index'] = data['g', 'walk', 'd']['edge_index'].to(device)
data['g', 'walk', 'c']['edge_index'] = data['g', 'walk', 'c']['edge_index'].to(device)
data['g', 'walk', 's']['edge_index'] = data['g', 'walk', 's']['edge_index'].to(device)

data['d', 'walk', 'g']['edge_index'] = data['d', 'walk', 'g']['edge_index'].to(device)
data['d', 'walk', 'd']['edge_index'] = data['d', 'walk', 'd']['edge_index'].to(device)
data['d', 'walk', 'c']['edge_index'] = data['d', 'walk', 'c']['edge_index'].to(device)
data['d', 'walk', 's']['edge_index'] = data['d', 'walk', 's']['edge_index'].to(device)

data['c', 'walk', 'g']['edge_index'] = data['c', 'walk', 'g']['edge_index'].to(device)
data['c', 'walk', 'd']['edge_index'] = data['c', 'walk', 'd']['edge_index'].to(device)
data['c', 'walk', 'c']['edge_index'] = data['c', 'walk', 'c']['edge_index'].to(device)
data['c', 'walk', 's']['edge_index'] = data['c', 'walk', 's']['edge_index'].to(device)

data['s', 'walk', 'g']['edge_index'] = data['s', 'walk', 'g']['edge_index'].to(device)
data['s', 'walk', 'd']['edge_index'] = data['s', 'walk', 'd']['edge_index'].to(device)
data['s', 'walk', 'c']['edge_index'] = data['s', 'walk', 'c']['edge_index'].to(device)
data['s', 'walk', 's']['edge_index'] = data['s', 'walk', 's']['edge_index'].to(device)

edge_indices = [ data['d', 'walk', 'g'].edge_index, data['d', 'walk', 'd'].edge_index, data['d', 'walk', 'c'].edge_index, data['d', 'walk', 's'].edge_index ]

tensor([[14763,     6],
        [19228,     6],
        [ 5145,     6],
        [16622,     6],
        [14749,     6],
        [ 5455,     6],
        [17735,     6],
        [10500,     6],
        [10694,     6],
        [ 8090,     6]])


In [4]:
args.portion = 0
args.im_ratio = [0.7, 0.1, 0.2, 0.2, 0.4, 0.4]
# args.device = torch.device("cpu")

In [5]:
c_train_num = dl.train_num(data['d'].y[:,1], args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['d'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, "\n", len(train_idx))
print("val_idx: ", val_idx, "\n", len(val_idx))
print("test_idx: ", test_idx, "\n", len(test_idx))
# print(c_num_mat)

[16, 16, 11, 2, 3, 3, 6, 6] 63
0 69
1 52
2 48
3 39
4 98
5 64
6 42
7 42
train_idx:  [588, 15505, 16964, 11569, 14473, 5988, 8879, 1608, 690, 17135, 6599, 8218, 3511, 4905, 3854, 3492, 11084, 18300, 3138, 7972, 16802, 11654, 10128, 212, 4095, 5075, 6634, 17316, 15382, 18173, 13076, 11772, 17576, 19875, 16875, 1706, 1780, 17601, 5716, 18945, 10595, 3427, 10518, 17825, 500, 13531, 7766, 5158, 1487, 9919, 16484, 26, 14613, 14798, 7701, 13110, 7449, 119, 18201, 16985, 12932, 19975, 5205] 
 63
val_idx:  [2372, 6352, 6123, 15486, 5431, 5618, 17751, 4186, 4579, 20038, 12056, 10192, 864, 2809, 8473, 4079, 4995, 12353, 8987, 5067, 9602, 20110, 18048, 2664, 16926, 9654, 13536, 5901, 9998, 5848, 12218, 4256, 18487, 19130, 14210, 8130, 4987, 4950, 7156, 12327, 14571, 2837, 1432, 1653, 16471, 14733, 12994, 15892, 3299, 2541, 11369, 11453, 13533, 17623, 16711, 8354, 17989, 13272, 11643, 425, 2148, 1854, 9572, 483, 8888, 19283, 4925, 348, 4241, 18702, 11251, 2205, 20093, 3035, 12484, 12314, 5491, 17783

In [6]:
### MAGNN
encoder1 = MAG_ConEn(args.embed_dim, args.dropout)
encoder2 = MAG_NetEn(args.embed_dim, args.dropout)
classifier = MAG_classify(args.embed_dim, args.nclass, args.dropout)
decoder_g = EdgePredictor(args.embed_dim)
decoder_d = EdgePredictor(args.embed_dim)
decoder_c = EdgePredictor(args.embed_dim)
decoder_s = EdgePredictor(args.embed_dim)
# decoder_g = torch.load('pretrained_han/decoder_g.pth')
# decoder_d = torch.load('pretrained_han/decoder_d.pth')
# decoder_c = torch.load('pretrained_han/decoder_c.pth')
# decoder_s = torch.load('pretrained_han/decoder_s.pth')

decoder_list = [decoder_g, decoder_d, decoder_c, decoder_s]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

In [7]:
### HETGNN

# encoder1 = Het_ConEn(args.embed_dim, args.dropout)
# encoder2 = Het_NetEn(args.embed_dim, args.dropout)
# classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
# decoder_a = EdgePredictor(args.embed_dim)
# decoder_p = EdgePredictor(args.embed_dim)
# decoder_v = EdgePredictor(args.embed_dim)
# # decoder_a = torch.load('pretrained/decoder_a.pth')
# # decoder_p = torch.load('pretrained/decoder_p.pth')
# # decoder_v = torch.load('pretrained/decoder_v.pth')
# decoder_list = [decoder_a, decoder_p, decoder_v]

# encoder1.to(device)
# encoder2.to(device)
# classifier.to(device)
# for decoder in decoder_list:
#     decoder.to(device)

## Training Part

In [8]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'recon')

[tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>)]
[tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>), tensor(0.0002, device='cuda:1', grad_fn=<DivBackward0>)]
[tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>), tensor(0.0002, device='cuda:1', grad_fn=<DivBackward0>), tensor(0.0003, device='cuda:1', grad_fn=<DivBackward0>)]
[tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>), tensor(0.0002, device='cuda:1', grad_fn=<DivBackward0>), tensor(0.0003, device='cuda:1', grad_fn=<DivBackward0>), tensor(2.8834e-05, device='cuda:1', grad_fn=<DivBackward0>)]
lossing
Epoch [1/200], Loss: 0.1237, Accuracy: 0.1250, Edge Accuracy: [0.9995005640246137, 0.9996456174741447, 0.9996660221980992, 0.9992973091865044]
class 0: 0.1, Class 0: AUC-ROC- 0.5424, F1 Score- 0.2222; class 1: 0.1, Class 1: AUC-ROC- 0.5552, F1 Score- 0.2222; class 2: 0.1, Class 2: AUC-ROC- 0.6021, F1 Score- 0.2222; class 3: 0.1, Class 3: AUC-ROC- 0.4537, F1 Score- 0.2222; class 4: 0.1, Class 4: AUC-ROC- 0.4157,

([0.11875,
  0.11875,
  0.11875,
  0.11875,
  0.11875,
  0.125,
  0.1125,
  0.1375,
  0.13125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.125,
  0.14375],
 [np.float64(0.46488839285714284),
  np.float64(0.46446428571428566),
  np.float64(0.4646874999999999),
  np.float64(0.4638169642857143),
  np.float64(0.4620982142857143),
  np.float64(0.46004464285714286),
  np.float64(0.4596875),
  np.float64(0.4587946428571429),
  np.float64(0.4602008928571429),
  np.float64(0.46185267857142853),
  np.float64(0.46314732142857146),
  np.float64(0.4639285714285714),
  np.float64(0.4647321428571428),
  np.float64(0.4639955357142857),
  np.float64(0.46037946428571425),
  np.float64(0.4580133928571429),
  np.float64(0.4598214285714286),
  np.float64(0.46189732142857143),
  np.float64(0.4635714285714285),
  np.float64(0.4641071428571429),
  np.float64(0.4669196428571428)],
 [np.float64(0.2222222222222222),
  np.float64(0.2222222222222222),
  np.float64(

In [ ]:
# torch.save(decoder_list[0], '../pretrained_magnn/decoder_g.pth')
# torch.save(decoder_list[1], '../pretrained_magnn/decoder_d.pth')
# torch.save(decoder_list[2], '../pretrained_magnn/decoder_c.pth')
# torch.save(decoder_list[3], '../pretrained_magnn/decoder_s.pth')
# torch.save(encoder1, '../pretrained_magnn/encoder1.pth')
# torch.save(encoder2, '../pretrained_magnn/encoder2.pth')